## LangChain Weather Agent with HeatWave GenAI

This notebook demonstrates how to build and run a simple, self-contained [LangChain](https://github.com/langchain-ai/langchain) agent that acts as a "weather bot." The agent is powered by a Large Language Model (LLM) running on MySQL [HeatWave GenAI](https://www.oracle.com/heatwave/genai/).

To orchestrate this, we will use a ZeroShotReActDescription agent. This type of agent uses the LLM's reasoning ability to determine a sequence of steps to solve a problem. It follows a "Thought -> Action -> Observation" loop, where it thinks about what to do, chooses a tool to act, and observes the result before deciding on the next step. The agent's reasoning is powered by a custom MyLLM wrapper, which allows LangChain to directly call LLMs hosted in MySQL HeatWave.

The overall goal is to create an agent that can answer the question, "What is the current weather in Fahrenheit?". To do this, the agent needs to perform a multi-step task:

- First, it will use a Weather tool to fetch the current weather conditions for its location. This tool gets the temperature in Celsius.
- Next, because the original request was for Fahrenheit, the agent will reason that a conversion is necessary. It will then use a Calculator tool to convert the Celsius temperature to Fahrenheit.

This demonstrates how tools can quickly enable a LLM to query live data (the weather tool can be easily replaced with a query to a live database), and how LLMs can use multi-step reasoning to solve complex problems. It also shows how HeatWave GenAI can be easily integrated into any agentic flow created with LangChain.

**This requires mysql-connector-python>=9.5.0**

### Connect to the HeatWave instance
First, we need to establish a connection to an active MySQL HeatWave instance on OCI.

**Action Required**: Create an SSH tunnel to your HeatWave instance by running the command below in your terminal, substituting the placeholder values with your OCI credentials.

ssh -o ServerAliveInterval=60 -i BASTION_PKEY -L LOCAL_PORT:DBSYSTEM_IP:DBSYSTEM_PORT BASTION_USER@BASTION_IP 

Modify the Python variables below to match the credentials for your HeatWave instance.

In [1]:
import mysql.connector

DBSYSTEM_SCHEMA = "ml_benchmark"
LOCAL_PORT = 3306  # The local port used in your SSH tunnel command

mydb = mysql.connector.connect(
    host="127.0.0.1",
    port=LOCAL_PORT,
    user="root",
    password="",
    database=DBSYSTEM_SCHEMA,
    allow_local_infile=True,
    use_pure=True,
    autocommit=True,
)
mycursor = mydb.cursor()

### Create a Weather tool to get the current weather
This function gets the machine's approximate location based on its IP address and then calls the Open-Meteo API to retrieve current weather conditions. The output is a structured dictionary containing location and weather data.

In [2]:
import requests
from langchain.agents import Tool


def get_current_location_and_weather(input_str: str = ""):
    # 1) Get approximate location via IP
    loc_resp = requests.get("https://ipapi.co/json/", timeout=5)
    loc_resp.raise_for_status()
    loc = loc_resp.json()

    lat = loc.get("latitude")
    lon = loc.get("longitude")
    if lat is None or lon is None:
        raise ValueError("Could not determine latitude/longitude from IP.")

    # 2) Get current weather from Open-Meteo
    params = {
        "latitude": lat,
        "longitude": lon,
        "current": "temperature_2m,apparent_temperature,relative_humidity_2m,wind_speed_10m,weather_code",
        "timezone": "auto",
    }
    wx_resp = requests.get(
        "https://api.open-meteo.com/v1/forecast", params=params, timeout=5
    )
    wx_resp.raise_for_status()
    wx = wx_resp.json().get("current", {})

    return {
        "location": {
            "city": loc.get("city"),
            "region": loc.get("region"),
            "country": loc.get("country_name"),
            "country_code": loc.get("country_code"),
            "latitude": lat,
            "longitude": lon,
            "ip": loc.get("ip"),
            "source": "ipapi.co (IP-based, approximate)",
        },
        "weather": {
            "time": wx.get("time"),
            "temperature_c": wx.get("temperature_2m"),
            "feels_like_c": wx.get("apparent_temperature"),
            "humidity_percent": wx.get("relative_humidity_2m"),
            "wind_speed_mps": wx.get("wind_speed_10m"),
            "weather_code": wx.get("weather_code"),
            "source": "open-meteo.com",
        },
    }


# Create the weather tool
weather_tool_object = Tool(
    name="Weather",
    func=get_current_location_and_weather,
    description=(
        "Gets the weather in Celsius for the current location. Has no inputs. When choosing this tool, always provide: Action Input:  (an empty string)."
    ),
)

### Create a Calculator tool to perform math calculations
This is a simple tool that takes a string containing a Python math expression, evaluates it, and returns the result. The agent will use this to perform the Celsius to Fahrenheit conversion.

In [3]:
def math_tool(input_str: str) -> str:
    # Defines a simple math tool
    try:
        return str(eval(input_str))
    except Exception:
        return "Sorry, I couldn't compute that."


# Runs the loop for a MySQL based math agent
math_tool_object = Tool(
    name="Calculator",
    func=math_tool,
    description="Useful for doing math calculations. Input should be a valid Python math expression.",
)

### Create an agent with the above tools
Now we assemble the agent. We provide it with the tools we created and bind it to the HeatWave LLM using the MyLLM wrapper. We also enable verbose=True to see the agent's step-by-step reasoning process

In [4]:
from langchain.agents import initialize_agent, AgentType
from mysql.ai.genai import MyLLM

# Bind defaults so they're always passed to the model's _call
my_llm = MyLLM(mydb).bind(model_id="meta.llama-3.3-70b-instruct")

agent = initialize_agent(
    tools=[math_tool_object, weather_tool_object],
    llm=my_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    max_iterations=2,  # prevents loops
    early_stopping_method="generate",  # produce best-effort Final Answer if stuck
    verbose=True,
)

/tmp/ipykernel_1661380/2560945101.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


### Ask the agent a question
Finally, let's ask our agent the question. Watch the output to see how it first calls the Weather tool, inspects the result, and then calls the Calculator tool to get the final answer.

In [5]:
agent.invoke({"input": "What is the current weather in Fahrenheit?"})



> Entering new AgentExecutor chain...
To find the current weather in Fahrenheit, I first need to get the current weather in Celsius and then convert it to Fahrenheit. 

Action: Weather
Action Input: 
Observation: {'location': {'city': 'Redwood City', 'region': 'California', 'country': 'United States', 'country_code': 'US', 'latitude': 37.5331, 'longitude': -122.2486, 'ip': '148.87.23.10', 'source': 'ipapi.co (IP-based, approximate)'}, 'weather': {'time': '2025-10-03T13:15', 'temperature_c': 19.6, 'feels_like_c': 19.2, 'humidity_percent': 60, 'wind_speed_mps': 6.6, 'weather_code': 1, 'source': 'open-meteo.com'}}
Thought:To convert the temperature from Celsius to Fahrenheit, I can use the formula: Fahrenheit = (Celsius * 9/5) + 32. I will use the Calculator tool to perform this calculation.

Action: Calculator
Action Input: (19.6 * 9/5) + 32
Observation: 67.28
Thought:Thought: I now know the final answer
Final Answer: The current weather is 67.28°F.

> Finished chain.


{'input': 'What is the current weather in Fahrenheit?',
 'output': 'The current weather is 67.28°F.'}

We invite you to try [HeatWave AutoML and GenAI](https://www.oracle.com/heatwave/free/). If you’re new to Oracle Cloud Infrastructure, try Oracle Cloud Free Trial, a free 30-day trial with US$300 in credits.